In [1]:
import pandas as pd
invoice_df = pd.read_csv('Downloads/fct_invoice.csv')
customer_df = pd.read_json('Downloads/dim_customer.json')

invoice_df.head() 

,invoice_no,customer_id,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,229,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,3433,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,644,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,4900,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,1089,Books,4,60.60,Cash,24/10/2021,Kanyon


In [2]:
customer_df.head() 

,age,first_name,gender,id,last_name
0,48.0,Nicholas,M,0,Flores
1,NaN,Jeffery,M,1,Rowe
2,57.0,Alexis,F,2,Benton
3,73.0,Amy,F,3,Johnson
4,20.0,David,M,4,Moore


1. How many unique customers are in the dataset?

In [3]:
customers = invoice_df['customer_id']

print(customers.nunique())

5191


2. What are the different categories of products available? How many unique categories are there?

In [4]:
categories = invoice_df['category']

print(categories.unique())
print(categories.nunique())

['Clothing' 'Shoes' 'Books' 'Cosmetics' 'Food & Beverage' 'Toys'
 'Technology' 'Souvenir']
8


3. Which payment method is the most popular? How many times was it used?

In [5]:
payment = invoice_df['payment_method'].value_counts().nlargest(1)

print(payment)

Cash    44447
Name: payment_method, dtype: int64


1. What are the three most popular categories, by total sales?

In [6]:
invoice_df['sales'] = invoice_df['price'] * invoice_df['quantity']
#get sales for each row

total_sales = invoice_df.groupby('category')['sales'].sum().nlargest(3)
#get total sales for each category and get top 3 categories

print(total_sales)

category
Clothing      1.139968e+08
Shoes         6.655345e+07
Technology    5.786235e+07
Name: sales, dtype: float64


2. What are the total sales attributed to customers over the age of 45?

In [7]:
invoice_df['sales'] = invoice_df['price'] * invoice_df['quantity']

total_sales = invoice_df.loc[customer_df['age'] > 45, 'sales'].sum()

print(total_sales)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

3. How is the data distributed across different invoice dates? Are there any seasonal trends or patterns? (Use a graph for this.)

In [ ]:
invoice_df['invoice_date'] = pd.to_datetime(invoice_df['invoice_date'])
import matplotlib.pyplot as plt

invoice_df['invoice_date'] = pd.to_datetime(invoice_df['invoice_date'])
#convert 'invoice_date' column to datetime format
invoice_df['month_year'] = invoice_df['invoice_date'].dt.to_period('M')
#extract month and year from 'invoice_date' column
monthly_counts = invoice_df['month_year'].value_counts().sort_index()
#count number of invoices for each month

monthly_counts.plot(kind='line', marker='o')
plt.title('Distribution of Invoices Over Time')
plt.xlabel('Month and Year')
plt.ylabel('Number of Invoices')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

1. Create a pivot table showing the breakdown of sales across these dimensions, in this order: category, decade age range (e.g., 10-19, 20-29, and so forth).

In [ ]:
merged_df = pd.merge(customer_df, invoice_df, left_on='id', right_on='customer_id')
#merge age and sales data based on customer_id column

merged_df['decade'] = pd.cut(merged_df['age'], bins=range(10, 101, 10), right=False)
#create new column for decade age range

pivot_table = pd.pivot_table(merged_df, values='price', index='category', columns='decade', aggfunc='sum')
#pivot table showing breakdown of sales by category and decade age range

print(pivot_table)